**We recommend to view the file at **

# Formatting and cleaning

# [1- Load basic modules and the csv file](#1)

# [2- Importing functions from Cleaning_functions.ipynb](#2)

# [3- Categorical variables](#3)

- ## [3.1- Gender](#3.1)
- ## [3.2- Cholesterol, Gluc, Smoke, Alco, Active](#3.2)

# [4- Continuous variables](#4)
- ## [4.1- Blood Pressure](#4.1)
- ## [4.2- Age](#4.2)
- ## [4.3- Height](#4.3)
- ## [4.4- Weight](#4.4)
- ## [4.5 BMI](#4.5)
- ## [4.6 Framingham Score](#4.6)
- ## [4.7 European Score](#4.7)

# [5- Saving to data_clean.csv](#5)

# [6- Saving to data_to_model.csv](#6)

<a id='1'></a>

---

## 1. Load basic modules and the csv file 

In [4]:
import pandas as pd
import numpy as np
import random as rd

data = pd.read_csv('../data_raw/Health_Screening_Data.csv')
data.drop(columns=['Unnamed: 0', 'id'],inplace=True)

In [5]:
data.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,AgeinYr,BMI,BMICat,AgeGroup
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,50,22.0,Normal,40-60
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,55,34.9,Obese,40-60
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,51,23.5,Normal,40-60
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,48,28.7,Over Weight,40-60
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,47,23.0,Normal,40-60


<a id='2'></a>

---

## 2. Importing functions from Cleaning_functions.ipynb

In [6]:
import sys
sys.path.append('../src/')
import Cleaning_functions as cf

<a id='3'></a>

---

## 3. Categorical variables

<a id='3.1'></a>

### 3.1 Gender

**The "gender" column is categorized into "f" and "m".**

In [7]:
data[["gender"]].head(5)

,gender
0,2
1,1
2,1
3,2
4,1


**The correction is made to categorize 1='m' and 2='f'**

In [8]:
data = cf.correct_gender(data)

In [9]:
data.gender.value_counts()

m    45502
f    24458
Name: gender, dtype: int64

<a id='3.2'></a>

### 3.2 Cholesterol, Gluc, Smoke, Alco, Active

**The  variables are coded as {0 1 2 3} as the case may be.**

In [10]:
categorical_variables = ["cholesterol", "gluc", "smoke", "alco", "active"]
data[categorical_variables].head()

,cholesterol,gluc,smoke,alco,active
0,1,1,0,0,1
1,3,1,0,0,1
2,3,1,0,0,0
3,1,1,0,0,1
4,1,1,0,0,0


**The value of the categorical variables is mapped to their corresponding values.**

In [11]:
info = {'cholesterol': {1:'normal', 2:'bordering', 3:'alto'},
        'gluc': {1:'normal', 2: 'prediabetes', 3: 'diabetes'},
        'smoke':{1:'yes', 0:'no'},
        'alco':{1:'yes', 0:'no'},
        'active':{1:'yes', 0:'no'}}

In [12]:
data = cf.correct_categorical(data)

In [13]:
categorical_variables = ["cholesterol", "gluc", "smoke", "alco", "active"]
data[categorical_variables].head()

,cholesterol,gluc,smoke,alco,active
0,normal,normal,no,no,yes
1,high,normal,no,no,yes
2,high,normal,no,no,no
3,normal,normal,no,no,yes
4,normal,normal,no,no,no


<a id='4'></a>

---

## 4. Continuous variables

<a id='4.1'></a>

### 4.1 Blood Pressure

**Abnormal systolic blood pressure values are corrected and imputed.**

In [14]:
data = cf.systolic(data, "ap_hi", "ap_lo")

**Diastolic pressure values containing between 4 and 5 digits are eliminated due to excess zeros (they are interpreted as typing errors).**

In [15]:
data = cf.remove_zeros(data, "ap_lo")

**Abnormal dyastolic blood pressure values are corrected and imputed.**

In [16]:
data = cf.dyastolic(data, "ap_hi", "ap_lo")

**Systolic blood pressure should be higher than diastolic. A check is made to ensure that this condition is met. If it is not met, then the values are reversed.**

In [17]:
data = cf.check_bp(data, "ap_hi", "ap_lo")

**We consider the upper limit of diastolic pressure to be 110. Diastolic pressure values greater than 110 will be replaced by 110.**

In [18]:
data = cf.dyastolic_final(data, "ap_lo")

**We find NaN values in ap_hi variable, we decide to erase those rows.**

In [19]:
data.dropna(inplace = True) 

<a id='4.2'></a>

### 4.2 Age

**The "age" and "AgeGroup" columns are removed as they are not considered useful. In turn, the "AgeCat" column is generated.**

In [20]:
data = cf.correct_age(data)

In [21]:
data[["AgeinYr", "AgeCat"]].head()

,AgeinYr,AgeCat
0,50,"[50, 60)"
1,55,"[50, 60)"
2,51,"[50, 60)"
3,48,"[40, 50)"
4,47,"[40, 50)"


<a id='4.3'></a>

### 4.3 Height

**Removing outliers from the "height" column.**

In [22]:
print(data["height"].max())
print(data["height"].min())

250
55


In [23]:
data = cf.remove_outliers(data, "height", 150, 200)

Shape of the initial dataframe: (69949, 15)
Shape of the dataframe after removing outliers: (67359, 15)


In [24]:
print(data["height"].max())
print(data["height"].min())

198
151


<a id='4.4'></a>

### 4.4 Weight

In [25]:
print(data["weight"].max())
print(data["weight"].min())

200.0
10.0


In [26]:
data = cf.correct_weight(data)

In [27]:
print(data["weight"].max())
print(data["weight"].min())

200.0
50.0


<a id='4.5'></a>

### 4.5 BMI

**Since the weight and height columns were corrected, the BMI variable is recalculated as well as its categorization**

In [28]:
data = cf.create_BMI(data)

In [29]:
data[["bmi", "bmi_cat"]].head()

,bmi,bmi_cat
0,21.97,Normal Weight
1,34.93,Obesity
2,23.51,Normal Weight
3,28.71,Overweight
4,23.01,Normal Weight


<a id='4.6'></a>

### 4.6 Framingham Score

In [30]:
def framingham(row):
    if row['gender'] == 'm':
        return cf.framingham_men(row)
    else:
        return cf.framingham_women(row)

In [31]:
data['fram'] = data.apply(framingham, axis=1)

<a id='4.7'></a>

### 4.7 European Score

In [32]:
table = cf.create_table_ESC()
data['esc'] = data.apply(cf.esc_score, args = (table,), axis = 1)

<a id='5'></a>

---

In [34]:
data

,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,AgeinYr,AgeCat,bmi,bmi_cat,fram,esc
0,f,168,62.0,110.0,80,normal,normal,no,no,yes,0,50,"[50, 60)",21.97,Normal Weight,15,2
1,m,156,85.0,140.0,90,high,normal,no,no,yes,1,55,"[50, 60)",34.93,Obesity,16,10
2,m,165,64.0,130.0,70,high,normal,no,no,no,1,51,"[50, 60)",23.51,Normal Weight,12,6
3,f,169,82.0,150.0,100,normal,normal,no,no,yes,1,48,"[40, 50)",28.71,Overweight,15,2
4,m,156,56.0,100.0,60,normal,normal,no,no,no,0,47,"[40, 50)",23.01,Normal Weight,11,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67354,f,168,76.0,120.0,80,normal,normal,yes,no,yes,0,52,"[50, 60)",26.93,Overweight,16,5
67355,m,158,126.0,140.0,90,bordering,prediabetes,no,no,yes,1,61,"[60, 70)",50.47,Obesity,14,11
67356,f,183,105.0,180.0,90,high,normal,no,yes,no,1,52,"[50, 60)",31.35,Obesity,16,5
67357,m,163,72.0,135.0,80,normal,prediabetes,no,no,no,1,61,"[60, 70)",27.10,Overweight,15,8


## 5. Saving to data_clean.csv

In [39]:
data[["AgeCat", "bmi_cat"]] = data[["AgeCat", "bmi_cat"]].astype("object")
data.to_csv('../documents/data_clean.csv', index=False)

<a id='6'></a>

---

## 6. Saving to data_to_model.csv

**The columns that will not be used as a source of information for the models are eliminated and data_to_model.csv is generated**

In [35]:
data_model = data.drop(columns=['height', 'weight', 'AgeCat', 'bmi_cat'])
data_model.head()
data_model.to_csv('../documents/data_to_model.csv', index=False)